# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM
from datasets import load_dataset
import numpy as np
import evaluate
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification


- Prepare a dataset

In [31]:
dataset = load_dataset("scene_parse_150", split="train[:150]")

/Users/hsin-wenchang/Documents/GitHub/Apply-Lightweight-Fine-Tuning-to-a-Foundation-Model/.env/lib/python3.9/site-packages/datasets/load.py:1454: FutureWarning: The repository for scene_parse_150 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scene_parse_150
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


- Tokenizer to process the text include a padding and truncation strategy to handle any variable sequence lengths. Using Hugging Face's datasets map method to apply a preprocessing function over the entire dataset:

In [28]:
tokenizer = AutoTokenizer.from_pretrained("abacusai/Smaug-72B-v0.1")



def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/28388 [00:00<?, ? examples/s]


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

- Create a smaller subset of the full dataset to fine-tune on to reduce the time it takes

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

- Loading model and specify the number of expected labels.

In [3]:
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=11)
model = AutoModelForCausalLM.from_pretrained("abacusai/Smaug-34B-v0.1", num_labels=11)

config.json:   0%|          | 0.00/753 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/85.5k [00:00<?, ?B/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00002-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model-00028-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00029-of-00030.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00030-of-00030.safetensors:   0%|          | 0.00/2.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

: 

- Create a TrainingArguments class which contains all the hyperparameters we can tune as well as flags for activating different training options then specify where to save the checkpoints from training

In [11]:
training_args = TrainingArguments(output_dir="test_trainer")

- Evaluate

In [12]:
metric = evaluate.load("accuracy")

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

- Create a Trainer object

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

- Fine-tune model by calling train():

In [15]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [04:10<00:00,  1.49it/s]

{'train_runtime': 250.9049, 'train_samples_per_second': 11.957, 'train_steps_per_second': 1.495, 'train_loss': 1.1278955078125, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=1.1278955078125, metrics={'train_runtime': 250.9049, 'train_samples_per_second': 11.957, 'train_steps_per_second': 1.495, 'train_loss': 1.1278955078125, 'epoch': 3.0})

In [16]:
trainer.evaluate()

100%|██████████| 125/125 [00:22<00:00,  5.57it/s]


{'eval_loss': 0.824751615524292,
 'eval_accuracy': 0.749,
 'eval_runtime': 22.6106,
 'eval_samples_per_second': 44.227,
 'eval_steps_per_second': 5.528,
 'epoch': 3.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

- Creating a PEFT Config

In [17]:
config = LoraConfig()

- Converting a Transformers Model into a PEFT Model
- Training with a PEFT Model And Checking Trainable Parameters of a PEFT Model

In [18]:
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 108,613,643 || trainable%: 0.27152390054719


In [19]:
training_args = TrainingArguments(
    output_dir="perft_trainer"
)

In [20]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

100%|██████████| 375/375 [02:46<00:00,  2.25it/s]

{'train_runtime': 166.85, 'train_samples_per_second': 17.98, 'train_steps_per_second': 2.248, 'train_loss': 0.38767240397135416, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.38767240397135416, metrics={'train_runtime': 166.85, 'train_samples_per_second': 17.98, 'train_steps_per_second': 2.248, 'train_loss': 0.38767240397135416, 'epoch': 3.0})

In [22]:
trainer.evaluate()

100%|██████████| 125/125 [00:20<00:00,  6.01it/s]


{'eval_runtime': 20.9832,
 'eval_samples_per_second': 47.657,
 'eval_steps_per_second': 5.957,
 'epoch': 3.0}

- Saving a Trained PEFT Model

In [23]:
lora_model.save_pretrained("bert-base-cased-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

- Loading a Saved PEFT Model

In [24]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("bert-base-cased-lora")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Generating Text from a PEFT Model

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
inputs = tokenizer("Hello, my name is ", return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
print(tokenizer.batch_decode(outputs))

TypeError: The current model class (BertForSequenceClassification) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BertLMHeadModel'}